# Conor Sayres
# Astro 507
# Homework 1

## 1a)

The code is implemented in C++ using pybind11 as a Python wrapper.  I chose C++ to allow me to experiment with small timesteps and large ensembles of particles while minimizing computation time. To build, type: 

```python setup.py build```

at the terminal to create the Python-importable module **cParticle**.

### File Descriptions:

**setup.py** - script to build the code

**cParticle.cpp** - Python bindings for the C++ Box class (simulator).

**particles.h** - C++ header.  Shows available classes, attributes and methods with some description.

**particles.cpp** - Implements the simulation routines.

### Code Description:

CGS units are used throughout the code. I model particles in a 2 dimensional box with reflecting edges. Particles are specified by the following initial conditions: mass, radius, x position, y position, x velocity, y velocity.  A box is defined by a width, height, and random seed. The Box class presents a method for easily adding Particles with random positions and directions at a specified velocity.  When a particle's position is within a radius of a wall, the velocity component normal to the wall is inverted. Reflections are tracked throughout the simulation to monitor pressure. Particles interact with one another via 2D elastic collisions. A collision is flagged by the following criteria:

1) The spacing between particles is less than or equal to the sum of the particles' radii.

2) The particles are moving towards eachother.

Enforcing criterion 2) improves the simulation results.  This protects against issues related to the time-stepping nature of the routine in which a collision may have recently occured (and momentum/energy transfered) and begun spatially separating, but not yet to the extent satisfying criterion 1).  Collision detection code is implemented in the Box::isCollided method.  Collision instances are modeled elastically.  To accomplish this I used the recipe specified here: 

https://en.wikipedia.org/wiki/Elastic_collision#Two-dimensional_collision_with_two_moving_objects

These equations are implemented in the Box::handleCollision method, which modifies the velocities of two particles experiencing a collision. The simulation itself proceeds as following:

1) Specify number of iterations (steps), and associated timestep (dt), and a rate at which to save the simulation state (saveEvery)

2) Begin loop over steps

3) At each step, modify all particles' positions forward in time (eg, x = x + vx*dt)

4) Reflect velocities for those particles at a boundary at this step

5) Check for collisions among all pairs of particles

6) For those particles collided, exchange momentum and energy elastically

7) Save the current state of the simulation (at specified frequency) for analysis later.

Here is a demonstration of a simulation of two particles in a box with unequal masses (progress during simulation is printed to terminal):

In [4]:
from cParticle import Box

width = 100
height = 100
mass = 1
radius = 10
velocity = width / 5
seed = 0
steps = 10000
dt = 0.001 * radius / velocity
saveEvery = 1000

box = Box(width, height, seed)
box.addRandomParticle(mass, radius, velocity)
box.addRandomParticle(10*mass, radius, velocity)
box.runSim(steps, dt, saveEvery)